In [ ]:
### Download the dataset from kaggle source
### you can find other videos in this link: https://www.kaggle.com/competitions/deepfake-detection-challenge/data
!wget -x --load-cookies cookies.txt -p www.kaggle.com/c/16880/datadownload/dfdc_train_part_45.zip 

--2020-03-27 04:26:46--  http://www.kaggle.com/c/16880/datadownload/dfdc_train_part_45.zip
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.kaggle.com/c/16880/datadownload/dfdc_train_part_45.zip [following]
--2020-03-27 04:26:46--  https://www.kaggle.com/c/16880/datadownload/dfdc_train_part_45.zip
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_part_45.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1585542406&Signature=OYjrHna%2BxoxUZIRCOtY1FQPHp64Wu8TKfxw3VVoleGfm%2FJ%2Fn32MVV8h1TH%2BJTLshQN7mKo80VhrSswnkbnwLhXMJmgqgcucF5bGxJ5FlTm1W4CbY8UORr9Rk4umi4i6lfFFdVxcyYhCMbyEfJl1YJxbAqIdB7YnXoeHdg4KkavIPk6pO0DnTJhIIKb4SW2BUOZzjozQTSXQK706HY

In [ ]:
! unzip -qq www.kaggle.com/c/16880/datadownload/dfdc_train_part_45.zip 

Archive:  www.kaggle.com/c/16880/datadownload/dfdc_train_part_45.zip
  inflating: dfdc_train_part_45/aaeflzzhvy.mp4  
  inflating: dfdc_train_part_45/aaeucwtkdx.mp4  
  inflating: dfdc_train_part_45/aaivbtdyjn.mp4  
  inflating: dfdc_train_part_45/aawddjkkoz.mp4  
  inflating: dfdc_train_part_45/aayyqrkmow.mp4  
  inflating: dfdc_train_part_45/abwlafapkf.mp4  
  inflating: dfdc_train_part_45/acjwdlpzob.mp4  
  inflating: dfdc_train_part_45/acjwolqjgw.mp4  
  inflating: dfdc_train_part_45/acovzzymlu.mp4  
  inflating: dfdc_train_part_45/acwegmlcgs.mp4  
  inflating: dfdc_train_part_45/adsxwxeibr.mp4  
  inflating: dfdc_train_part_45/adxkyxkepg.mp4  
  inflating: dfdc_train_part_45/aeepmwyhkk.mp4  
  inflating: dfdc_train_part_45/aefpqqxvbe.mp4  
  inflating: dfdc_train_part_45/aevqxxxhno.mp4  
  inflating: dfdc_train_part_45/aevxvajusj.mp4  
  inflating: dfdc_train_part_45/aexkdsocpo.mp4  
  inflating: dfdc_train_part_45/afolgdwiix.mp4  
  inflating: dfdc_train_part_45/afqatecrhd.mp4  


In [ ]:
!pip install mtcnn     ### no matter what face detector you use. 
!rm -r www.kaggle.com  ### for saving the space

     |████████████████████████████████| 2.3MB 4.8MB/s 


In [ ]:
import os
import cv2
import json
from random import randint, shuffle
import tensorflow as tf
import numpy as np
from mtcnn import MTCNN

detector = MTCNN()
num_of_reals = 1
num_of_fakes = 1
c = 1
frame_number= 0
dirs = ["dfdc_train_part_45"]

output_dir = 'faces/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for dir in dirs:
  m = 0 
  files = os.listdir(dir)
  shuffle(files)
  print('Starting ... num of vids in this dir is:', len(files))
  with open( os.path.join(dir , 'metadata.json')) as f:
    data = json.load(f)
    for key , values in data.items():
      video_name = key
      video_label = values['label']
      for file in files:
        if file == video_name:
          passing_frames = 5   ### skip frames
          m += 1 
          print(file + '    '+str(m) + ' of ' + str(len(files)))
          cap = cv2.VideoCapture(os.path.join(dir,file))
          while(cap.isOpened()):
            ret , frame = cap.read()
            frame_number += 1
            if ret :
              if (frame_number%passing_frames) ==0:
                detected_faces_raw = detector.detect_faces(frame)
                for face in detected_faces_raw:
                  x,y,w,h=face['box']
                  confidence = face['confidence']
                  if confidence > 0.96:
                    try:
                      x1 = x - 40
                      y1 = y - 40
                      w1 = w + 80
                      h1 = h + 110
                      if x<0:
                        x=0
                      if y<=0:
                        y=0
                      final_frame = (cv2.resize(frame[y1:y1+h1,x1:x1+w1],(256,256)))
                    except:
                      final_frame = (cv2.resize(frame[y:y+h,x:x+w],(256,256)))
                    frame_path = output_dir + video_label + '_' + str(c) + '.jpg'

                    if video_label == 'REAL':
                      print("done REAL",num_of_reals)
                      num_of_reals += 1
                      passing_frames = 15 ### different passing frames for balancing the dataset

                    elif video_label == 'FAKE':
                      print("done FAKE",num_of_fakes)
                      num_of_fakes += 1
                      passing_frames = 5

                    cv2.imwrite(frame_path , final_frame)
                    c = c+1
            else:
              break
          cap.release()
  print('dir' +dir+ ' done. founded ' + str(c) + ' faces. including ' + str(num_of_reals) + ' REAL faces and ' + str(num_of_fakes) + ' FAKE faces.')


Using TensorFlow backend.


Streaming output truncated to the last 5000 lines.
done FAKE 3750
done FAKE 3751
tchqfddubz.mp4    1463 of 2347
done FAKE 3752
done FAKE 3753
done FAKE 3754
varfagexso.mp4    1464 of 2347
done REAL 2267
done REAL 2268
done REAL 2269
done REAL 2270
done REAL 2271
done REAL 2272
done REAL 2273
done REAL 2274
done REAL 2275
done REAL 2276
hgyjtixhyn.mp4    1465 of 2347
done FAKE 3755
done FAKE 3756
done FAKE 3757
jgwuktnwgg.mp4    1466 of 2347
done FAKE 3758
done FAKE 3759
done FAKE 3760
bwabxrljfa.mp4    1467 of 2347
done FAKE 3761
done FAKE 3762
done FAKE 3763
fnbwfrgbam.mp4    1468 of 2347
done FAKE 3764
done FAKE 3765
done FAKE 3766
ryspsmtomm.mp4    1469 of 2347
done FAKE 3767
done FAKE 3768
done FAKE 3769
shrpamtirw.mp4    1470 of 2347
done REAL 2277
done REAL 2278
done REAL 2279
done REAL 2280
done REAL 2281
done REAL 2282
done REAL 2283
done REAL 2284
done REAL 2285
done REAL 2286
done REAL 2287
done REAL 2288
bpcmfykufo.mp4    1471 of 2347
done FAKE 3770
done FAKE 3771
done FAKE 